# Overview <a id=ov>

1. [Features](#features)
2. [Label Construction and Analysis](#labels)
3. [Feature Analysis (Mini)](#feature_ana)
4. [Model Definition and Training](#model)
5. [Training Visualization](#train_vis)

In [ ]:
import pandas as pd
from os import listdir
from copy import deepcopy
from os.path import isdir

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from itertools import cycle
from utils import retrieve_if_necessary

In [ ]:
def get_experiment_descriptor_fjc(fname):
    segs = fname.split('__')
    instance_name = segs[2].split('_')[0]
    seed = segs[2].split('_')[1][len('seeded'):]
    # print(instance_name, seed)
    return instance_name, seed

In [ ]:
%%time

csv_list = []

for fname in listdir(data_path):
    fp = f'{data_path}{fname}'
    if not isdir(fp):
        partial_df = pd.read_csv(fp, index_col=0)
        ename, seed = get_experiment_descriptor_fjc(fname)
        partial_df['benchmark_name'] = [ename] * partial_df.shape[0]
        partial_df['seed'] = int(seed)
        csv_list.append(partial_df)

In [ ]:
%%time

df_complete = pd.concat(csv_list)
print(df_complete.shape)

In [ ]:
seeds = np.unique(df_complete['seed'])

seeds.shape

In [ ]:
import sys

sys.path.append('../../../fabricatio-controls/')

from fabricatio_controls.comparison_utils import store_seeds

seed_fp = f'../1_data/seeds/1_analysis_seeds_{setup}.txt'

store_seeds(seeds, seed_fp)

# 1. Features <a id=features>
    
[Back to Overview](#ov)    

Somewhat too many columns. 400 out of them are the raw state inputs which we are not so interested in. These are prefixed with `op_t_`, `op_d_`, `op_l_`, `op_s_` for the type, location, duration and status matrices respectively. 

Let us drop these for a sec.

In [ ]:
column_subset = [c for c in df_complete.columns 
                 if c[:5] not in ['op_t_', 'op_d_', 'op_l_', 'op_s_']]


df_subset1 = df_complete[column_subset]

unique_vals = df_subset1.nunique()

pd.set_option('display.max_rows', unique_vals.shape[0]+1)
display(unique_vals)
print(len(unique_vals))

Of these remaining 111 columns, the ones starting with a heuristic name (e.g. `SRPT_`) followed by `NoneType_` and a KPI mark the achieved respective goal by the particular jeuristics.

Available goals are, as you can see, `makespan`, `tardiness`, `utilization`, `throughput_op` (i.e. operation throughput), `throughput_j` (i.e. job throughput) and `flow_time`. We will stick with the makespan for now. 

In [ ]:
cumulative_r_names = [
    'cum_r_util_ave_diff_continuous', 'cum_r_util_ave_diff_discrete',
    'cum_r_util_exp', 'cum_r_util_timescaled', 'cum_r_util_std_diff_discrete',
    'cum_r_makespan_continuous', 'cum_r_makespan_normed', 'cum_r_buff_len'
]

heuristic_names = [
    'SPT_LQT', 'LPT_LQT', 'LOR_LQT', 'MOR_LQT', 
    'SRPT_LQT', 'LRPT_LQT', 'LTPO_LQT', 
    'MTPO_LQT', 'EDD_LQT', 'LUDM_LQT']

col_subset2 = [c for c in df_subset1.columns 
               if ('_'.join(c.split('_')[:2]) not in heuristic_names 
               or (c.split('_')[-1] == 'makespan' or c.split('_')[2] == 'cum')) 
               or (c.split('_')[0] in heuristic_names 
               and c.split('_')[1] != 'NoneType')]

df_subset2 = df_complete[col_subset2]

unique_vals = df_subset2.nunique()

pd.set_option('display.max_rows', unique_vals.shape[0]+1)
display(unique_vals)
print(len(unique_vals))

Nice :)
    
Now, from these columns, `direct_action` and ``indirect_action`` cannot be used to predict the best heuristic/the makespan achieved by the best heuristic since these represent the decisions taken by the winning heuristic, i.e. they are basically the label. The dicrete field `benchmark_name` is also somethingour model cannot know beforehand. Hence we exclude these three columns as well.


**If you're wondering about the fields marked with the `r_` prefix, these are possible quantities used for *rewards* during the training of an RL scheduler and could very well be used as features, since they are available at every decision point. Note however, that the column should be shifted appropriately, since the reward $r_{t}$ is available only after a decision has been made given the state $s_{t}$. As such, for every partial dataframe, the reward columns should be shifted by one, such that $s_{t}$ is associated with $s_{t-1}$.** 

In [ ]:
col_subset3 = [c for c in df_subset2.columns 
               if c not in ["direct_action", "indirect_action", "benchmark_name"]]

df_subset3 = df_subset2[col_subset3]

unique_vals = df_subset3.nunique()

pd.set_option('display.max_rows', unique_vals.shape[0]+1)
display(unique_vals)
print(len(unique_vals))

## Feature Plots

In [ ]:
%%time

f_name = 'wip_to_arrival_ratio'

def mean_std(group_df, f_name):
    d = {}
    d[f'{f_name}_ave'] = group_df[f_name].mean()
    d[f'{f_name}_std'] = group_df[f_name].std()
    return pd.Series(d)

def get_step_plot_df(df_complete, f_name):
    return (df_complete[[f_name, 'n_steps']]
            .groupby('n_steps').apply(lambda x: mean_std(x, f_name) )
            .reset_index())

def get_time_plot_df(df_complete, f_name):
    """
    Creates the mean and standard deviation signals for the argument feature at the 
    distinct action times. To speed up the plotting process, the signals are resampled 
    every 100 time units and aggreggated using the mean.
    """
    df_feature = df_complete[['t_action', f_name]]
    line_rgr_vis_df = df_feature.sort_values('t_action').reset_index(drop=True)
    line_rgr_vis_df.index = pd.to_datetime(line_rgr_vis_df.index)
    std = line_rgr_vis_df.resample('100ns').std()
    means = line_rgr_vis_df.resample('100ns').mean()
    df_plot = means.copy()
    df_plot = df_plot.rename(columns={f_name: f'{f_name}_ave'})
    df_plot[f'{f_name}_std'] = std[f_name]
    return df_plot
    
df_plot = get_step_plot_df(df_complete, f_name)

In [ ]:
import seaborn as sns
import itertools
from matplotlib import pyplot as plt

sns.set_style('whitegrid')
sns.set_context('talk')
pal = itertools.cycle(sns.color_palette('cubehelix', 2))

def plot_feature_distribution(f_name, df_plot, ylabel='', x_axis='n_steps', x_label="Step"):
    _, ax = plt.subplots(1, 1, figsize=(5.5, 3.75))#, gridspec_kw={'height_ratios': [3, 1]})
    
    ax = sns.lineplot(data=df_plot, x=x_axis, y=f'{f_name}_ave', 
                 color=next(pal), ax=ax)
    ax.fill_between(df_plot[x_axis],
                     df_plot[f'{f_name}_ave'] - df_plot[f'{f_name}_std'], 
                     df_plot[f'{f_name}_ave'] + df_plot[f'{f_name}_std'], 
                     alpha=.5, color=next(pal))
    ax.set_xlabel(x_label)
    ax.set_ylabel(ylabel)
#     ax = sns.histplot(x=df_plot[f'{f_name}_ave'], kde=True, ax=ax[1])
#     ax.set_xlabel(ylabel)
    plt.tight_layout()
    plt.savefig(f"{plot_base}/51_fjc__{f_name}.png", dpi=200, bbox_inches='tight', pad_inches=0.02)
    plt.show()

# df_plot = get_step_plot_df(df_complete, 'wip_to_arrival_ratio')
# plot_feature_distribution('wip_to_arrival_ratio', df_plot, "WIP Fill")
# df_plot = get_step_plot_df(df_complete, 'utl_avg')
# plot_feature_distribution('utl_avg', df_plot, "Average System \nUtilization")
df_plot = get_time_plot_df(df_complete, 'wip_to_arrival_ratio')
plot_feature_distribution('wip_to_arrival_ratio', df_plot, x_axis="t_action", x_label="Time")
df_plot = get_time_plot_df(df_complete, 'utl_avg')
plot_feature_distribution('utl_avg', df_plot, x_axis="t_action", x_label="Time")

In [ ]:
f_names = [
    'buffer_length_ratio',                        
    'buffer_load_ratios_avg',                    
    'buffer_load_ratios_std',                    
    'buffer_time_ratio',                        
    'decision_skip_ratio',
    'duration_ave',                              
    'duration_distance_mean',                    
    'duration_distance_std',                    
    'duration_entropy',                          
    'duration_std',        
    'estimated_flow_time_ave',
    'estimated_flow_time_std',
    'estimated_tardiness_rate',                   
    'heuristic_agreement_entropy',                 
    'job_op_completion_rate_ave',                 
    'job_op_completion_rate_std',                 
    'job_op_max_rel_completion_rate_avg',         
    'job_op_max_rel_completion_rate_std',         
    'job_work_completion_rate_avg',              
    'job_work_completion_rate_std',              
    'job_work_max_rel_completion_rate_avg',      
    'job_work_max_rel_completion_rate_std',     
    'kendalltau_ave',                            
    'kendalltau_std',                            
    'legal_action_job_ratio',                       
    'legal_action_len_stream_ave',             
    'legal_action_len_stream_std',             
    'makespan_lb_ub_ratio',                    
    'op_completion_rate',                         
    'silhouette_kmeans_max_k',                    
    'silhouette_kmeans_mid_k',                   
    'silhouette_kmeans_min_k',                  
    'tardiness_rate',                              
    'throughput_time_j_abs_avg',                
    'throughput_time_j_abs_std',               
    'throughput_time_j_rel_avg',                
    'throughput_time_j_rel_std',                
    'type_ave',                                   
    'type_entropy',                              
    'type_hamming_mean',                         
    'type_hamming_std',                          
    'type_std',  
    'utl_current',
    'utl_avg',                                 
    'utl_std',                                 
    'wip_rel_sys_t',                           
    'wip_to_arrival_ratio',                         
    'wip_to_arrival_time_ratio',                  
    'work_completion_rate',    
]
len(f_names)

In [ ]:
import json

with open('fea_names.txt', 'w') as f:
    json.dump(f_names, f)

In [ ]:
#%%time

sns.set_style('whitegrid')
sns.set_context('talk')


def check_name_correspondence(fea_name_rank, df_tab):
    # preprocessing
    fea_name_rank = fea_name_rank.replace('utl', 'utilization')
    fea_name_rank = fea_name_rank.replace('std', 'standard_deviation')
    fea_name_rank = fea_name_rank.replace('avg', 'average')
    fea_name_rank = fea_name_rank.replace('_j_', '_job_')
    fea_name_rank = fea_name_rank.replace('job_work_max_rel', 'work_max_rel')
    fea_name_rank = fea_name_rank.replace('rel', 'relative')
    fea_name_rank = fea_name_rank.replace('abs', 'absolute')
    fea_name_rank = fea_name_rank.replace('len', 'length')
    # rank components
    scores = []
    names = []
    for name in df_tab['Name']:
        scores.append(0)
        names.append(name)
        searchstring = ''.join(name.lower().split()).replace('\\', '')
        segs = fea_name_rank.split('_')
        for seg in segs:
            if seg in searchstring: 
                scores[-1] += 1
    #print(scores)
    best_score_pos = np.argmax(scores)
    return names[best_score_pos], best_score_pos


def add_figure_head():
    tex_string = r'\begin{figure}[ht!]' + '\n'
    tex_string += r'\centering' + '\n'
    tex_string += r'\resizebox{\linewidth}{!}{' + '\n'
    tex_string += r'\begin{tabular}{@{}ll@{}}' + '\n'
    return tex_string


def add_figure_tail(setup_str, part):
    plt_title = setup_str + 'Feature Behavior (' + part + ').'
    tex_string = r'\end{tabular}}' + '\n'
    tex_string += (r'\caption[' + plt_title + r']{\small ' + plt_title +
                   r'}\label{fig:ex_instances}' + '\n')
    tex_string += r'\end{figure}' + '\n\n'
    return tex_string

def get_tex_string(f_name, f_display_name, i, setup='fjc'):
    tex_string = ''
    setup_string = r'$FJc$' if setup == 'fjc' else r'$Jm$'
    if (i % 8 == 0):
        if i != 0:
            tex_string += add_figure_tail(setup_string, f'{i-7}-{i}')                
        tex_string += add_figure_head()
    subfloat_caption = (f'\\subfloat[\n\t\\scriptsize {setup_string} '
                        f'{f_display_name} Behavior.')
    label = '\\label{subfig:' + setup + '_' + f_name + '}\n]'
    includegraphics = '{\n\t\\includegraphics[width=0.45\\linewidth]{'
    graphics_path = r'main_matter/images/x_' + setup + r'_' + f_name + r'.png}}'
    line_ending = '&\n' if i % 2 == 0 else '\\\\\n'
    return (tex_string + subfloat_caption + label + includegraphics + graphics_path + line_ending)

stp = 'fjc'
setup_string = r'$FJc$' if stp == 'fjc' else r'$Jm$'

df_tab = pd.read_csv('fea_tab.csv')
pal = itertools.cycle(sns.color_palette(
    'cubehelix', int(1.5 * len(f_names))))

tex_string = ''
i = 0
for f_name in f_names:
    print(f_name)
    if f_name == 'utl_avg':
        continue
    df_plot = get_time_plot_df(df_complete, f_name)
    col = next(pal)
    ax = sns.lineplot(data=df_plot, x='t_action', y=f'{f_name}_ave', color=col)
    ax.fill_between(df_plot['t_action'], df_plot[f'{f_name}_ave'] - df_plot[f'{f_name}_std'], df_plot[f'{f_name}_ave'] + df_plot[f'{f_name}_std'], alpha=.5, color=col)
    ax.set_xlabel('Simulation Time')
    display_name, idx = check_name_correspondence(f_name, df_tab)
    print(display_name)
    ax.set_ylabel('')
    tex_string += get_tex_string(f_name, display_name, i, stp)
    plt.tight_layout()
    plt.savefig(f'D://1_research//2_disertation//1_document//main_matter//images//x_{stp}_{f_name}.png', dpi=200, bbox_inches='tight', pad_inches=0.02)
    plt.show()
    i += 1

tex_string += add_figure_tail(setup_string, f'{(i // 8) * 8 + 1}-{i}')

In [ ]:
print(tex_string)

In [ ]:
%%time
n_cols = 4
n_rows = int(np.ceil(len(f_names) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(7 * n_cols, 10 * n_rows))#, 
                     #gridspec_kw={'height_ratios': [3, 1]})
pal = itertools.cycle(sns.color_palette('cubehelix', int(1.5 * len(f_names))))

i = 0
for f_name in f_names:
    df_plot = get_step_plot_df(df_complete, f_name) 
    col = next(pal)
    ax = sns.lineplot(data=df_plot, x='n_steps', y=f'{f_name}_ave', 
             color=col, ax=axes[i // 4][i % 4])
    ax.fill_between(df_plot['n_steps'],
                    df_plot[f'{f_name}_ave'] - df_plot[f'{f_name}_std'], 
                    df_plot[f'{f_name}_ave'] + df_plot[f'{f_name}_std'], 
                    alpha=.5, color=col)
    if i < (n_rows - 1) * n_cols - (n_cols - n_rows % 4):
        ax.set_xticklabels([])
        ax.set_xlabel('')
    else:
        ax.set_xlabel('Simulation Step')
    ax.set_ylabel(' '.join([x.capitalize() for  x in f_name.split('_')]))
    # ax[1] = sns.histplot(x=df_plot[f'{f_name}_ave'], kde=True, ax=ax[1])
    i += 1
    

while i < n_rows * n_cols:
    fig.delaxes(axes[i // 4][i % 4])
    i += 1

fig.tight_layout()
fig.subplots_adjust(hspace=0.2)

plt.savefig(f'feature_overview_lineplots_{setup}.png', dpi=200)
plt.show()

In [ ]:
%%time

n_rows = int(np.ceil(len(f_names) / 4))

fig, axes = plt.subplots(n_rows, 4, figsize=(28, 10 * n_rows))#, 
                     #gridspec_kw={'height_ratios': [3, 1]})
    
pal = itertools.cycle(sns.color_palette('cubehelix', int(1.5 * len(f_names))))

i = 0
for f_name in f_names:
    df_plot = get_step_plot_df(df_complete, f_name) 
    col = next(pal)
    ax = sns.histplot(x=df_plot[f'{f_name}_ave'].astype('float32'), kde=True, 
                      ax=axes[i // 4][i % 4], color=col)
    if i // 4 < 10:
        ax.set_xlabel('')
    ax.set_ylabel(' '.join([x.capitalize() for  x in f_name.split('_')]))
    
    i += 1

fig.delaxes(axes[i // 4][i % 4])
plt.tight_layout()
plt.savefig(f'feature_overview_histplots_{setup}.png', dpi=200)
plt.show()

## Label Construction

Now let us construct the classification/regression labels. For now we focus on AlphaZero style predictions meaning that we try and predict the state value in the case of regression and not the action value. For DQN style predictions we would need to do a simultaneous regression for all possible heuristics in the current state. A sicussion for another time ;)

For classification, we simply select the heuristic having achieved the best makespan for any given state. For regression, we select the makespan score corresponding to the latter.

To compute the labels, we have to: 
1. select the label fields
2. get the argmin of the row for the clf label
3. get the corresponding makespan value for the rgr label

This will take a few seconds given the 800k+ datapoints...

In [ ]:
df_subset3

In [ ]:
h_result_cols = [c for c in df_subset3.columns if c.endswith('makespan')]
h_result_cnames = [c.split('_')[0] for c in h_result_cols]
h_control_complete_names = ['_'.join(c.split('_')[:2]) for c in h_result_cols]

cr_cols = []

#list1_permutations = itertools.permutations(list1, len(list2))
for cname in h_control_complete_names:
    for rname in cumulative_r_names:
        cr_cols.append(f'{cname}_{rname}')

# display(all_combinations)
# df_subset3[~df_subset3.isin(f_names)]

In [ ]:
for col in h_result_cols:
    try:
        assert (df_subset3['t_action'] > df_subset3[col]).sum() == 0
    except AssertionError:
        print(col)

In [ ]:
%%time

# 1. select label fields
df_label_raw = df_subset3[h_result_cols + cr_cols]
# 2. get the best heuristic and its value
def get_labels(row, clf_names, h_control_complete_names):
    clf_label_encoded = np.argmin(row[h_result_cols])
    clf_label_name = clf_names[clf_label_encoded]
    pertinent_cumulative_reward_cols = [
        f'{h_control_complete_names[clf_label_encoded]}_{cr}' 
        for cr in cumulative_r_names]
    rgr_label = row[clf_label_encoded]
    # print(row[pertinent_cumulative_reward_cols])
    filtered_row = tuple([clf_label_encoded, clf_label_name, rgr_label]
                         + row[pertinent_cumulative_reward_cols].tolist())
    return filtered_row

label_series = df_label_raw.apply(
    lambda x: get_labels(x, h_result_cnames, h_control_complete_names), axis=1)
label_df = pd.DataFrame(
    label_series.values.tolist(), 
    columns=['clf_label', 'clf_label_name', 'rgr_label'] + cumulative_r_names)

In [ ]:
def convert_num_cols(df):
    for col in df.columns:
        if df[col].dtypes == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtypes == 'int64':
            df[col] = df[col].astype('int32')
        else:
            continue

# test_df = label_df.copy()
# print(test_df.dtypes)
# convert_num_cols(test_df)
# print(test_df.dtypes)

Make best makespan relative by subtracting wip start time then replace the heuristic result fields with the newly constructed labels

In [ ]:
norm_factor_df = df_complete[[c for c in df_complete.columns if c[:5] == 'op_d_']]
norm_factor = norm_factor_df.sum(axis=1)
# norm_factor_df

In [ ]:
df_subset3['t_action']

In [ ]:
label_df['rgr_s_label'] = (label_df['rgr_label'] - df_subset3['t_action'].values) 
label_df['rgr_rel_label'] = (label_df['rgr_s_label'] 
                             / norm_factor.values)

In [ ]:
%%time

# label_df.index = df_subset3.index

label_df = label_df.reset_index(drop=True)
df_final = df_subset3.drop(h_result_cols + cr_cols, axis=1).reset_index(drop=True)
convert_num_cols(label_df)
convert_num_cols(df_final)

for col in label_df.columns:
    df_final[col] = label_df[col]

unique_vals = df_final.nunique()

pd.set_option('display.max_rows', unique_vals.shape[0]+1)

display(unique_vals)
print(len(unique_vals))

In [ ]:
df_final.shape

All the features here are nummeric or ordinal so you're basically good to go :)

# 2. Label Construction and Visualization  <a id=labels>

[Back to Overview](#ov)

In [ ]:
# !pip install -U seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle

sns.set_context('talk')
sns.set_style('whitegrid')

def plot_heuristic_comparison(clf_label_list, figname=""):
    # TODO: consistent colors between heuristic names and 
    # corresponding bars between plots ;)
    palette = cycle(sns.color_palette('cubehelix', 2))
    names, values = np.unique(clf_label_list,return_counts=True)
    values = values / values.sum()
    green = next(palette)
    magenta = next(palette)
    colors = [green if values[i] > 0.05 else magenta
           for i in range(values.shape[0])]
    ax = sns.barplot(x=names, y=values, palette=colors)
    ax.set_xticklabels(names, rotation=45)
    # fig = plt.figure(figsize=(15, 8))
    # ax = sns.histplot(sorted(df_final['clf_label_name']), 
    #                   palette='cubehelix', discrete=True)
    #ax.set_xticklabels(h_result_cnames)
    ax.set_xlabel("Heuristic Name")
    ax.set_ylabel("Success Rate")
    plt.tight_layout()
    if figname:
        plt.savefig(f'{plot_base}/{figname}.png', 
                    dpi=200, bbox_inches='tight', pad_inches=0.02)

plot_heuristic_comparison(sorted(df_final['clf_label_name']), 
                          f"52_heuristic_distribution_{setup}")
plt.show()

In [ ]:
def filter_p_df(df_final):
    el, counts = np.unique(df_final['clf_label'], return_counts=True)
    counts_list = []       
    ps = counts / counts.sum()
    ave = int(counts.mean())
    subsamples = []
    for i in range(10):
        if i in el:
            idx = np.where(el == i)
        if ps[idx] <= 0.05:
            continue
        h_sample = df_final[df_final['clf_label'] == i]
        subsamples.append(h_sample)
    df_resampled = pd.concat(subsamples, axis=0)
    return df_resampled

In [ ]:
def filter_mean_df(df_final):
    el, counts = np.unique(df_final['clf_label'], return_counts=True)
    ave = int(counts.mean())
    subsamples = []
    for i in range(10):
        h_sample = df_final[df_final['clf_label'] == i]
        subsamples.append(h_sample.sample(min(ave, h_sample.shape[0])))
    df_resampled = pd.concat(subsamples, axis=0)
    return df_resampled

In [ ]:
%%time

df_resampled = filter_p_df(df_final)

# fig = plt.figure(figsize=(15, 8))
# ax = sns.histplot(sorted(df_resampled['clf_label_name']), 
#                   palette='cubehelix', discrete=True)

plot_heuristic_comparison(sorted(df_resampled['clf_label_name']), 
                          f"52_besth_distribution_{setup}")
# plt.show()

[DEPRECATED DESCRIPTION]

Before movin any further we need to take another look at the regression label, with which there is a problem: Because of the data gathering process, the makespan label shifted in time with each step. The makespan label always includes the time needed for job long gone fromm the wip, of wich our future model knows nearly nothing. We could assume, that using the field `t_start`, which indicates the system time at which the step was taken, a regression model could learn to approximate the correct heuristic results by simply adding this time to whatever prediction it would yiel. Nevertheless we can make the model's life easier by manually ajusting the regression labels relative to the step system time. 

We can then drop the baseline field. This has the added advantage that it makes heuristic results comparable over all the datapoints.

In [ ]:
df_resampled = df_final

In [ ]:
sns.set_context('poster')
sns.set_style('whitegrid')

h_names = ['SPT', 'LPT', 'LOR', 'MOR', 'SRPT', 'LRPT', 'LTPO', 'MTPO', 'EDD', 'LUDM']
fig = plt.figure(figsize=(15, 8))
ax = sns.boxplot(x=df_resampled['clf_label'], y=df_resampled['rgr_label'], 
                 order=list(range(len(h_names))))
ax.set_xticklabels(h_names)
ax.set_title("Distribution of Heuristic Makespan in 'Win Cases'", loc='left')
ax.set_xlabel("Heuristic")
ax.set_ylabel("'Win' Makespan") 
plt.tight_layout()
plt.savefig(f'{plot_base}/when_do_heuristics_win_{setup}.png', dpi=300)
plt.show()

In [ ]:
line_rgr_vis_df = df_resampled.copy().sort_values('t_action').reset_index(drop=True)

line_rgr_vis_df.index = pd.to_datetime(line_rgr_vis_df.index)
std = line_rgr_vis_df.resample('100ns').std()
means = line_rgr_vis_df.resample('100ns').mean()

In [ ]:
%%time

fig = plt.figure(figsize=(15, 8))
sns.lineplot(data=means, x='t_action', y='rgr_label')

plt.fill_between(means['t_action'],
                 means['rgr_label'] - std['rgr_label'], 
                 means['rgr_label'] + std['rgr_label'], alpha=.3)
plt.show()

In [ ]:
sns.set_context('poster')
sns.set_style('whitegrid')

h_names = ['SPT', 'LPT', 'LOR', 'MOR', 'SRPT', 'LRPT', 'LTPO', 'MTPO', 'EDD', 'LUDM']
fig = plt.figure(figsize=(15, 8))
ax = sns.boxplot(x=df_resampled['clf_label'], y=df_resampled['rgr_s_label'], 
                 order=list(range(len(h_names))))
ax.set_xticklabels(h_names)
ax.set_title("Distribution of WIP Start Relative Heuristic Makespan in 'Win Cases'", 
             loc='left')
ax.set_xlabel("Heuristic")
ax.set_ylabel("'Win' Makespan") 
plt.tight_layout()
plt.savefig(f'{plot_base}/when_do_heuristics_win.png', dpi=300)
plt.show()

In [ ]:
assert (df_final['rgr_s_label'] > 0).all()

In [ ]:
%%time

fig = plt.figure(figsize=(15, 8))
sns.lineplot(data=means, x='t_action', y='rgr_s_label')
plt.fill_between(means['t_action'],
                 means['rgr_s_label'] - std['rgr_s_label'], 
                 means['rgr_s_label'] + std['rgr_s_label'], alpha=.3)
plt.show()

In [ ]:
sns.set_context('poster')
sns.set_style('whitegrid')

h_names = ['SPT', 'LPT', 'LOR', 'MOR', 'SRPT', 'LRPT', 'LTPO', 'MTPO', 'EDD', 'LUDM']
fig = plt.figure(figsize=(15, 8))
ax = sns.boxplot(x=df_resampled['clf_label'], y=df_resampled['rgr_rel_label'], 
                 order=list(range(len(h_names))))
ax.set_xticklabels(h_names)
ax.set_title("Distribution of Start and Duration Relative "
             "Heuristic Makespan in 'Win Cases'", loc='left')
ax.set_xlabel("Heuristic")
ax.set_ylabel("'Win' Makespan") 
plt.tight_layout()
plt.savefig(f'{plot_base}/when_do_heuristics_win.png', dpi=300)
plt.show()

In [ ]:
%%time

fig = plt.figure(figsize=(15, 8))
sns.lineplot(data=means, x='t_action', y='rgr_rel_label')
plt.fill_between(means['t_action'],
                 means['rgr_rel_label'] - std['rgr_rel_label'], 
                 means['rgr_rel_label'] + std['rgr_rel_label'], alpha=.3)
plt.show()

### Resource Utilization Over Decision Time

In [ ]:
%%time

fig = plt.figure(figsize=(15, 8))
utl_ave = means['utl_avg'] 
sns.lineplot(x=means['t_action'], y=utl_ave)
plt.fill_between(means['t_action'],
                 means['utl_avg'] - std['utl_std'], 
                 means['utl_avg'] + std['utl_std'], alpha=.3)
plt.show()

In [ ]:
predictors = f_names

In [ ]:
r_cols = [c for c in df_resampled.columns if c.startswith('r_')]

groups = (df_final[r_cols + ['n_steps', 'seed']]
          .sort_values(['seed', 'n_steps'])
          .groupby(['seed'])
          .cumsum())
# groups

In [ ]:
df_r_analysis_full = df_final.sort_values(['seed', 'n_steps'])
df_r_analysis_full[r_cols] = groups[r_cols]

In [ ]:
df_predictor_test = df_final.dropna()
print(df_predictor_test.shape)

In [ ]:
df_r_analysis_end = (df_r_analysis_full[df_r_analysis_full.columns]
                     .sort_values(['seed', 'n_steps'])
                     .groupby(['seed']).last())
# df_r_analysis_end

# 3. Feature Analysis (Mini) <a id=feature_ana>

[Back to Overview](#ov)

Some correlations:

In [ ]:
# %%time

# _ = plt.figure(figsize=(60, 30))

# sns.heatmap(df_resampled.corr().abs(), 
#             linewidths=.5, annot=True, vmin=0.1, fmt='1.2f', 
#             vmax=1, 
#             cmap='cubehelix')
# plt.tight_layout()
# plt.savefig(f'feature_correlation_{setup}.png', dpi=300)
# plt.show()

Let us take a cloaser look at the correlation between the cumulative reward and the regression labels:

In [ ]:
def plot_reward_correlations(df_cum_r, cr_cols, figname_suffix=''):
    l_cols = [
        'rgr_label', 
#         'rgr_s_label', 
#         'rgr_rel_label'
    ]
    df_r = df_cum_r[l_cols + cr_cols]
    df_corr = df_r.corr()
    names = {
        "rgr_label": "T: Makespan",
#         "rgr_s_label": "T2: End-WIP \nStart-Relative \nMakespan",
#         "rgr_rel_label": "T3: End-WIP \nStart-Relative \nNormed Makespan",
        "cum_r_util_ave_diff_continuous": "R1: Utilization \nContinuous Difference",
        "cum_r_util_ave_diff_discrete": "R2: Utilization \nQuantized Difference",
        "cum_r_util_exp": "R3: Utilization \nSigmoid",
        "cum_r_util_timescaled": "R4: Timescaled \nUtilization Difference",
        "cum_r_util_std_diff_discrete": "R5: Utilization \nQuantized Deviation",
        "cum_r_makespan_continuous": "R6: Negative \nAbsolute Makespan",
        "cum_r_makespan_normed": "R7: Duration \nRelative Makespan",
        "cum_r_buff_len": "R8: Negative \nCumulative Buffer Length"
    }
    df_corr.rename(columns = names, index = names, inplace=True)
    #df_corr.rename() = df_corr.columns.copy()
    _ = plt.figure(figsize=(15, 13))

    sns.set_context('poster')
    sns.heatmap(df_corr, 
                linewidths=.5, annot=True, vmin=-1, fmt='1.2f', 
                vmax=1, 
                cmap='cubehelix')
    plt.tight_layout()
    if figname_suffix:
        plt.savefig(f'{plot_base}/53_reward_correlation_{setup}_{figname_suffix}.png', dpi=300, 
                    bbox_inches='tight', pad_inches=0.02)
    plt.show()
    
def get_reward_choice(df_cum_r, cr_cols, target_label, rank_position):
    l_cols = [
        'rgr_label', 
        'rgr_s_label', 
        'rgr_rel_label'
    ]
    df_r = df_cum_r[l_cols + cr_cols]
    df_corr = df_r.corr()
    df_corr.loc[target_label, 
                ['rgr_label', 'rgr_rel_label', 'rgr_s_label']] = [0, 0, 0]
    return df_corr.columns[
        np.where(
            sorted(df_corr.loc[target_label,:])[rank_position - 1] 
            == df_corr.loc[target_label,:])].values[0]

The three regression labels are all important in different ways. The normed action time relative label emphasizes "winns" for individual wip configurations whitout considering by how much. The action time relative label emphasizes the magnitude of the makespan for individual wip windows. Last but not least, the "raw" regression label relativizes the curent success with respect to the past. 

Since we are dealing with an online problem, we want our reward estimate the overall makespan as well as possible. Hence, we choose the reward that best correlates with the overall makespan as our candidate.

In [ ]:
r_cols

In [ ]:
cr_cols = [f'cum_{c}' for c in r_cols]
plot_reward_correlations(df_r_analysis_full, cr_cols, figname_suffix='full')

print(get_reward_choice(df_r_analysis_full, cr_cols, 'rgr_label', 1))
print(get_reward_choice(df_r_analysis_full, cr_cols, 'rgr_s_label', 1))
print(get_reward_choice(df_r_analysis_full, cr_cols, 'rgr_rel_label', 1))

plot_reward_correlations(df_r_analysis_end, cr_cols)
print(get_reward_choice(df_r_analysis_end, cr_cols, 'rgr_label', 1))
print(get_reward_choice(df_r_analysis_end, cr_cols, 'rgr_s_label', 1))
print(get_reward_choice(df_r_analysis_end, cr_cols, 'rgr_rel_label', 1))

In [ ]:
from itertools import cycle

_, axes = plt.subplots(4, 2, figsize=(15, 13))


palette = cycle(sns.color_palette('cubehelix', 12))
for i in range(len(cr_cols)):
    sns.histplot(df_r_analysis_full[cr_cols[i]], 
                 kde=True, ax=axes[i // 2][i % 2], 
                 color=next(palette), bins=30)

plt.tight_layout()
plt.savefig(f"{plot_base}/reward_distributions.png")
plt.show()

In [ ]:
l_cols = [
    'rgr_label', 
    'rgr_s_label', 
    'rgr_rel_label'
]

_, axes = plt.subplots(1, 3, figsize=(15, 4))

palette = cycle(sns.color_palette('cubehelix', 4))
for i in range(len(l_cols)):
    sns.histplot(df_r_analysis_full[l_cols[i]], 
                 kde=True, ax=axes[i % 3], color=next(palette), )

plt.tight_layout()
plt.savefig(f"{plot_base}/label_distributions.png")
plt.show()

In [ ]:
reward_choice = get_reward_choice(df_r_analysis_full, cr_cols, 'rgr_label', 1)
reward_choice

In [ ]:
df_predictor_test = df_r_analysis_full.dropna()
print(df_predictor_test.shape)
fin_cpm = df_final.dropna()
print(fin_cpm.shape)

Let us see how well the selected reward can be predcted, and which features are most important :)

In [ ]:
from typing import List
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_validate, train_test_split

def get_feature_rankings(cv_results: dict, predictors: List[str], scoring="MAE"):
    """
    Extracts feature rankings from a cv results object containing a trained 
    random forest model.
    """
    # get feature rankings
    feature_scores = []
    for estimator in cv_results['estimator']:
        feature_scores.append(estimator.feature_importances_)

    df_feature_importance = pd.DataFrame({
        'feature_name': predictors,
        'feature_importance': np.stack(feature_scores).mean(axis=0)
    })
    print(f"Cross-Validated {scoring}: {cv_results['test_score'].mean()}")
    return df_feature_importance.sort_values(
        'feature_importance', ascending=False).reset_index(drop=True)

def train_random_forest_ranker(predictors: List[str], label: str, clf: bool = False):
    seeds_tried = []
    top10 = None
    i = 1
    while True:   
        seed = 58629 # np.random.choice(90000, 1)[0]
        print(seed)
        seeds_tried.append(seed)
        if clf==True:
            feature_ranker = RandomForestClassifier()
            scoring = 'accuracy'
        else:
            feature_ranker = RandomForestRegressor(random_state=seed)
            scoring = "neg_mean_absolute_error"
        df_predictor_test_sample = df_predictor_test.sample(10000, random_state=seed)
        cv_results = cross_validate(feature_ranker, 
                                    df_predictor_test_sample[predictors + ['t_action']], 
                                    df_predictor_test_sample[label],
                                    cv=10, scoring=scoring, 
                                    return_estimator=True, 
                                    return_train_score=True, n_jobs=4)
        top10 = get_feature_rankings(cv_results, predictors + ['t_action'])['feature_name'].values[:11]
        print(top10)
        break
    return cv_results

# 81784
# ['job_work_completion_rate_avg' 'kendalltau_ave' 'type_hamming_std'
#  'duration_distance_std' 'job_work_completion_rate_std'
#  'type_hamming_mean' 'duration_distance_mean' 'op_completion_rate'
#  'job_op_completion_rate_ave' 'duration_ave']

In [ ]:
%%time

estimator_container_abs_l = train_random_forest_ranker(predictors, 'rgr_label')
f_importance_abs_l_df = get_feature_rankings(estimator_container_abs_l, predictors + ['t_action'])

In [ ]:
f_importance_abs_l_df = f_importance_abs_l_df[~(f_importance_abs_l_df['feature_name'] == 't_action')]
f_importance_abs_l_df.to_csv('5_feature_importance_fjc.csv')
f_importance_abs_l_df

In [ ]:
# fi_jm = pd.read_csv('5_feature_importance_jm.csv', index_col=0)
# fi_fjc = pd.read_csv('5_feature_importance_fjc.csv', index_col=0)


# def get_rl_model_features_overlap(df_fi):
#     model_features = {'legal_action_len_stream_ave', 'tardiness_rate',
#                       'legal_action_len_stream_std', 'wip_rel_sys_t',
#                       'decision_skip_ratio', 'type_hamming_mean',
#                       'duration_distance_mean', 'wip_to_arrival_ratio',
#                       'estimated_tardiness_rate', 'estimated_flow_time_ave'}
#     overlap = model_features.intersection(df_fi['feature_name'].values[:10])
#     return overlap


# def compute_rank(fi_df):
#     rank = 1
#     score = 1
#     fi_df['rank'] = np.zeros(fi_jm.shape[0])
#     for i in range(1, fi_df.shape[0]):
#         if fi_df['feature_importance'][i] < score:
#             # print(score)
#             fi_df['rank'][i] = rank
#             rank += 1
#             score = fi_df['feature_importance'][i]
#         elif fi_df['feature_importance'][i] == score:
#             fi_df['rank'][i] = rank
#     return fi_df
        
# fi_jm = fi_jm.sort_values('feature_importance', ascending=False)
# fi_jm = compute_rank(fi_jm)
# fi_fjc = fi_fjc.sort_values('feature_importance', ascending=False)
# fi_fjc = compute_rank(fi_fjc)


# fi_overall = fi_fjc.join(fi_jm.set_index('feature_name'), 
#                          on='feature_name', how='inner', lsuffix='_fjc', rsuffix='_jm')
# fi_ov_sorted = fi_overall.sort_values(
#     'rank_fjc', ascending=True).reset_index(drop=True)

# fi_ov_sorted

# print("Jm&FJc Features Overlap: ")
# overlap_fi_ov = get_rl_model_features_overlap(fi_ov_sorted)
# print(len(overlap_fi_ov), overlap_fi_ov)

# fi_ov_sorted

Classification Results:

In [ ]:
%%time

estimator_container_clf = train_random_forest_ranker(predictors, 'clf_label', clf=True)
f_importance_clf_df = get_feature_rankings(estimator_container_clf, predictors)

overlap_clf = get_rl_model_features_overlap(f_importance_clf_df)
print(len(overlap_clf), overlap_clf)

In [ ]:
print("Top 10 regression")
top10_regr_df = df_feature_importance_clf.sort_values(
    'feature_importance', ascending=False)[:10]
print(top10_regr_df.to_latex())
display(top10_regr_df['feature_name'].values)

### Standard Deviation of Heuristic Decision Scores Over Decision Time

In [ ]:
df_subset3.columns

In [ ]:
%%time

avg_std_ddf = df_label_raw.copy()
# get normed label variance per row
# for col in avg_std_ddf.columns:
#     avg_std_ddf[col] -= df_subset3['t_action'].values
#     avg_std_ddf[col] /= norm_factor.values
    
avg_std_ddf['t_action'] = df_subset3['t_action'].values

# thin data
line_label_mstd_df = deepcopy(avg_std_ddf).sort_values(
    't_action').reset_index(drop=True)

line_label_mstd_df.index = pd.to_datetime(line_label_mstd_df.index)
std = line_label_mstd_df.resample('100ns').std()
means = line_label_mstd_df.resample('100ns').mean()

# plot
sns.set_context('poster')
sns.set_style('whitegrid')

_, axes = plt.subplots(2, 1, 
                       figsize=(15, 20), 
                       gridspec_kw={'height_ratios': [3, 1]})
means_no_start = means.drop(columns=['t_action'])

df_label_raw.std(axis=1)

fig = plt.figure(figsize=(20, 8))
sns.lineplot(data=means, 
             x='t_action', 
             y=means_no_start.mean(axis=1), ax=axes[0], 
             label='Average Decision Scores')

axes[0].fill_between(
    means['t_action'],
    means_no_start.mean(axis=1) - means_no_start.std(axis=1), 
    means_no_start.mean(axis=1) + means_no_start.std(axis=1), 
    color='red',
    alpha=.3, label='Standard Deviation')

sns.lineplot(data=means, 
             x='t_action', 
             y=means_no_start.std(axis=1), ax=axes[1])

axes[0].set_ylabel('Decision Score \n'
                   '(Makespan Relative to Duration Sum)')
axes[0].set_xlabel('')
axes[1].set_ylabel('Decision Score\n Standard Deviation')
axes[1].set_xlabel('State Time')
plt.show()